# Quantum Unstructured Search Algorithm (Grover's Algorithm)


Some useful resources for understanding the implementation of Grover's algorithm:
https://www.youtube.com/watch?v=AabglvQSBR4&list=PL74Rel4IAsETUwZS_Se_P-fSEyEVQwni7&index=46 


## Problem Statement 

Given n input qubits (initialized to zero), Grover's algorithm will search an output an element in a "database". For example if you were looking for the element "11", in binary "3", Grover's algorithm will be able to find this element.  

In [30]:
# install requirements
import numpy as np
from numpy import pi
# importing Qiskit
from qiskit import QuantumCircuit, execute, Aer, IBMQ
from qiskit.providers.ibmq import least_busy
from qiskit.tools.monitor import job_monitor
from qiskit.visualization import plot_histogram, plot_bloch_multivector
import matplotlib.pyplot as plt
import numpy as np
from qiskit.quantum_info.operators import Operator
import operator
%config InlineBackend.figure_format = 'svg' # Makes the images look nice

# Algorithm Instructions

There are two parts to this algorithm (aside from initialization):

1. Amplitude Amplification
2. Rotation about the Mean (this part can be rotated a number of times in order to increase the probability that the target element is returned)


## Amplitude Amplification 

There are a few ways to implement this. One of those ways is to use the "Phase Oracle".

For this example, i'm going to for go the phase oracle and apply a unitary matrix directly on the initialized qubits. We can use this method because amplitude amplification can be represented by a diagonal matrix will all elements $A_{ij} \neq target$ can be 1, which means that the target will be -1. For example if my target is "11" (binary "3")

The amplitude amplification can be represented by the matrix:

$$\begin{bmatrix} 1 & 0 & 0 & 0 \\ 0 & 1 & 0 & 0 \\ 0 & 0 & 1 & 0 \\ 0 & 0 & 0 & -1 \end{bmatrix}$$


In [76]:
# test with 2 qubit example
def grover():
    n=3
    qc = QuantumCircuit(n, n)
    
    # initialize qubits with 
    for x in range(n):
        qc.h(x)
    
    for x in range(n):
        grover_iteration(qc, n)

    qc.measure(range(n), range(n))
    
    return qc


def grover_iteration(qc, n):
    
    # amplitude amplification
    oracle_matrix = np.identity(2**n)
    # add the -1 phase to marked elements
    oracle_matrix[5, 5] = -1

    # convert your matrix (called oracle_matrix) into an operator, and add it to the quantum circuit
    qc.unitary(Operator(oracle_matrix), range(n), label="amp amplf")
    
    # reflection about 0 basis vector 
    refl = np.full((2**n,2**n), 2/(2**n), float)
    refl[np.diag_indices_from(refl)] = (2/(2**n)) - 1
    qc.unitary(Operator(refl), range(n), label="reflection")
 
    
qc = grover()
qc.draw("mpl")


simulator = Aer.get_backend('qasm_simulator')
counts = execute(qc, backend=simulator, shots=1000).result().get_counts(qc)
from qiskit.visualization import plot_histogram
plot_histogram(counts)

SyntaxError: invalid syntax (<ipython-input-76-7c5ef082ecc9>, line 10)